In [1]:
import pymssql  
import pandas as np
import random as rdm
import datetime
import time
start = time.time()

In [2]:
def generate_insert(table,num,site):
    row = [ 'tagname','site','rs_v','st_v','tr_v','r_a','s_a','t_a','kwh','pf','kw','is_on','datetime']
    
    if(rdm.randint(0,100)>50):
        ison = 1
    else:
        ison = 0
    
    datas = {
        'tagname' : "A%s"%(str(num)),
        'site' : "N"+str(site),
        'rs_v' : 380,
        'st_v' : 380,
        'tr_v' : 380,
        'r_a' : rdm.randint(0,1000),
        's_a' : rdm.randint(0,1000),
        't_a' : rdm.randint(0,1000),
        'kwh' : rdm.randint(0,1000),
        'pf' : rdm.randint(0,1000),
        'kw' : rdm.randint(0,1000),
        'is_on' : ison,
        'datetime' : datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    template = "insert into %s ("%(table)
    template = template+','.join(row)+") values ("
    for col in row:
        template = template+"'"+str(datas[col])+"'"+','
    template = template[:-1]+')'
    return template

In [8]:
generate_insert('point_info',5,1)

"insert into point_info (tagname,site,rs_v,st_v,tr_v,r_a,s_a,t_a,kwh,pf,kw,is_on,datetime) values ('A5','N1','380','380','380','264','365','289','641','194','884','0','2019-08-16 15:54:27')"

In [3]:
def simulate_sql(db,cursor,table,points):
    for num in range(points):
        try:
            data = generate_insert(table,num+1,1)
            cursor.execute(data)
            db.commit()
        except :
            db.rollback()
            print("error occurs")
            return 0
    return 1

In [7]:
#db = pymssql.connect(server='localhost:1401', user='SA', password='Samj6t/6bj6', database='testDB')
db = pymssql.connect(server='192.168.50.45:1433', user='app', password='1qaz2wsx', database='app_project')  
# 使用 cursor() 方法创建一个游标对象 cursor
cursor = db.cursor()
# 使用 fetchone() 方法获取单条数据.
period = 10
count = 0
day = 7
point_num = 9
start = time.time()
value = day*86400/period*point_num
result = 1
limit = 500
print("point_num : %d , sleep time : %d , start time : %f , limit : %d" %(point_num,period,start,limit))
#print("%d days , %d points , there will be %d datas will be insert"%(day,point_num,day*86400/period*point_num))
while count<(day*86400/period):
    #print("Number %d data incoming."%(count+1))
    time.sleep(period)
    result = simulate_sql(db,cursor,'point_info',point_num)
    if not result:
        print("error occurs")
        break
    if count > limit:
        break
    count+=1
# 关闭数据库连接
end = time.time()
print('Insert finish , cost time : %d'%(end-start))
db.close()

point_num : 9 , sleep time : 10 , start time : 1566545288.593089 , limit : 500


KeyboardInterrupt: 